In [60]:
# A simple predict the next character by looking at last k characters

In [1]:
!wget https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/indic-corp-frozen-for-the-paper-oct-2022/mr.txt

--2024-07-08 16:08:50--  https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/indic-corp-frozen-for-the-paper-oct-2022/mr.txt
Resolving objectstore.e2enetworks.net (objectstore.e2enetworks.net)... 164.52.210.97, 101.53.152.33, 164.52.206.154, ...
Connecting to objectstore.e2enetworks.net (objectstore.e2enetworks.net)|164.52.210.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14553660884 (14G) [text/plain]
Saving to: ‘mr.txt’

mr.txt                0%[                    ]       0  --.-KB/s               ^C


In [2]:
# Load the dataset

k = 50000
input_file_path = './data/mr.txt'
output_file_path = f"./data/mr_{k}.txt"

# Function to read the first k lines from the input file and write them to the output file
def read_and_write_first_k_lines(input_file, output_file, num_lines=1000):
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            for i in range(num_lines):
                line = infile.readline()
                if not line:  # End of file reached before 1000 lines
                    break
                outfile.write(line)
        print(f"Successfully wrote the first {num_lines} lines to {output_file}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
read_and_write_first_k_lines(input_file_path, output_file_path, k)

data_file = output_file_path
with open(data_file, 'r') as file:
    lines = file.readlines()

Successfully wrote the first 50000 lines to ./data/mr_50000.txt.


In [55]:
# Let's build a vocabulary
context = 3
vocab = set()
for line in lines:
    if line.strip() != "":
        # words = line.split(" ")
        # Remove blank words
        # words = [word.strip() for word in words if word.strip() != "" ]
        line = '<sos>' + line.strip() + '<eos>'
        for ch in line:
            vocab.add(ch)
vocab = list(vocab)
vocab_size = len(vocab)
print(vocab_size)

335


In [30]:
word_to_i = {word: i for i, word in enumerate(vocab)}
i_to_word = {i: word for word, i in word_to_i.items()}
print(word_to_i['.'])

216


In [47]:
# Let's get the training dataset
from collections import deque

xs = []
ys = []

def get_queue():
    qu = deque(maxlen=context)
    for _ in range(context):
        qu.append('.')
    return qu

for line in lines:
    qu = get_queue()
    if line.strip() != "":
        line = line.strip()
        qu.append(line[0])
        for ch in line[1:]:
            xs.append(list(qu))
            ys.append(ch)
            qu.append(ch)
print(lines[0])
print(len(xs))
# for x, y in zip(xs, ys):
#     print(x, y)

ऊती संवर्धन तंत्राचे अनेक उपयोग आहेत. या तंत्राचा उपयोग विशेषकरून जीवशास्त्र व वैद्यकशास्त्रात होतो. वयोवृद्धी, पोषण, लसनिर्मिती, जन्मजात रोगांचे निदान, इंद्रियांचे रोपण, कर्करोग संशोधन व गर्भपोषण या क्षेत्रांत ऊती संवर्धन तंत्र प्रामुख्याने वापरले जाते. पेशींच्या चयापचयावर एखाद्या घटकाचा परिणाम पाहणे, सामान्य किंवा कर्करोगाच्या पेशींवर औषधांचा होणारा परिणाम पाहणे, प्रयोगशाळेत त्वचा तयार करणे इ. बाबी ऊती संवर्धनामुळे शक्य झाल्या आहेत. भाजलेल्या रुग्णाच्या त्वचारोपणासाठी ऊती संवर्धनाद्वारे निर्माण केलेली त्वचा वापरली जाते.

7132887


In [51]:
import torch

xs_enc = []
ys_enc = []
for inp, out in zip(xs, ys):
    temp = []
    for i in inp:
        temp.append(word_to_i[i])
    xs_enc.append(temp)
    ys_enc.append(word_to_i[out])
xs_enc = torch.tensor(xs_enc)
ys_enc = torch.tensor(ys_enc)

In [50]:
print(xs_enc.shape)
print(ys_enc.shape)
num_examples = len(xs)

torch.Size([7132887, 3])
torch.Size([7132887])


In [57]:
import torch.nn.functional as F

w = torch.randn((vocab_size, vocab_size), requires_grad=True)
xs_one_hot = F.one_hot(xs_enc, num_classes=vocab_size).float()

In [58]:
# This should be 7132887 * 3 * 335
xs_one_hot.shape

torch.Size([7132887, 3, 335])